# Иследование продаж видео игр

В моём распоряжении данные о продажах игр, оценки пользователей и экспертов, жанры и платформы. Необходимо выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

У меня есть данные до 2016 года. Представим, что сейчас декабрь 2016 г., и я планирую кампанию на 2017-й. Данные за 2016 год могут быть неполными.

Цель исследования:
- изучить информацию о продажах игр.
- выявить определяющие успешность игры закономерности.
- проверить гипотезы, которые помогут понять, какие игры стоит продвигать в 2017 году.
- сделать ставку на потенциально популярный продукт, чтобы спланировать рекламные кампании.

Ход исследования:

Необходимые данные получу из датасета games.csv. О состоянии и качестве данных ничего не известно, поэтому я сначала открою файл и изучу общую информацию.

Предобработаю данные:
- Если будут проблемы с наименованием столбцов, то переименую их, чтобы привести к удобным и читаемым названиям.
- Проверю на явные дубликаты и если такие имеются, то удалю.
- Проверю на наличие пропусков и обработаю их.
- Если будет необходимость, то исправлю типы данных
- Обработаю неявные дубликаты.
- Для удобного анализа в дальнейшем - добавлю необходимые столбцы.

Перейду к исследовательскому анализу данных, где опишу и визуализирую информацию о продажах игр:
- Посмотрю, сколько игр выпускалось в разные годы.
- Посмотрю, как менялись продажи по платформам.
- Возьму данные за соответствующий актуальный период.
- Изучу какие платформы лидируют по продажам, растут или падают.
- Построю график "ящик с усами" по глобальным продажам игр в разбивке по платформам.
- Посмотрю, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков.
- Соотнесу выводы с продажами игр на других платформах.
- Посмотрю на общее распределение игр по жанрам.

Составлю портрет пользователя каждого региона:
- Определю для пользователя каждого региона (NA, EU, JP):
    - Самые популярные платформы (топ-5).
    - Самые популярные жанры (топ-5).
    - Влияет ли рейтинг ESRB на продажи в отдельном регионе?

После этого проверю гипотезы:
- Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.
- Средние пользовательские рейтинги жанров Action и Sports разные.

В завершении сделаю общий вывод по исследованию.

## Открою файл с данными и изучу общую информацию

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st

In [2]:
data = pd.read_csv('./datasets/games.csv')

Посмотрю на первые 5 строк датафрейма и посмотрю общую информацию о данных.

In [3]:
data.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Я смог прочесть датасет, каких-то проблем при этом не возникло.

В датасете 11 столбцов и 16715 строк. В некоторых столбцах есть пропуски и неподходящие типы данных. Названия столбцов необходимо привести к нижнему регистру.

## Предобработка данных

### Переименование столбцов

Приведём названия столбцов к стандарту snake_case - в нашем случае это только нижний регистр, так как андерскоры уже есть в названиях столбцов.

In [5]:
data.columns = data.columns.str.lower()

Проверю, что названия столбцов изменились.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


### Обработка пропусков

Проверю на наличие явных дубликатов.

In [7]:
data.duplicated().sum()

0

Определю, сколько пропусков в каждом столбце.

In [8]:
# Создам функцию для определения пропусков, чтобы не писать одно и то же несколько раз
def missing_values(data_with_na):
    return pd.DataFrame(data_with_na.isna().mean() * 100).style.format("{:.2f}%").background_gradient('coolwarm')

In [9]:
missing_values(data)

,0
name,0.01%
platform,0.00%
year_of_release,1.61%
genre,0.01%
na_sales,0.00%
eu_sales,0.00%
jp_sales,0.00%
other_sales,0.00%
critic_score,51.32%
user_score,40.09%


В столбцах name, year_of_release и genre есть небольшое количество пропусков - 0.01%, 1.61% и 0.01% соответственно.
В целом, их можно удалить ничего сильно от этого не изменится.
В critic_score, user_score, rating пропусков уже больше - 51.32%, 40.09% и 40.48% соответственно.
Удалять их уже нельзя, так как это большая часть данных. Нужно будет посмотреть на данные детально, чтобы понять, что с ними можно сделать.

Удалю пропуски в столбцах name, year_of_release и genre.

In [10]:
data = data.dropna(subset=['name', 'year_of_release', 'genre'])

Проверю, что пропуски удалились.

In [11]:
missing_values(data)

,0
name,0.00%
platform,0.00%
year_of_release,0.00%
genre,0.00%
na_sales,0.00%
eu_sales,0.00%
jp_sales,0.00%
other_sales,0.00%
critic_score,51.45%
user_score,40.17%


Заполню пропуски в critic_score медианным значением рейтинга критиков по каждой игре.

In [12]:
data['critic_score'] = data['critic_score'].fillna(data.groupby('name')['critic_score'].transform('median'))

Проверю, остались ли после этого пропуски в critic_score.

In [13]:
missing_values(data)

,0
name,0.00%
platform,0.00%
year_of_release,0.00%
genre,0.00%
na_sales,0.00%
eu_sales,0.00%
jp_sales,0.00%
other_sales,0.00%
critic_score,45.74%
user_score,40.17%


Доля пропусков уменьшилась на \~6% - значит, что для остальных игр нет данных о рейтинге критиков.
Позже, когда буду анализировать данные, посмотрю, какие игры остались без рейтинга критиков и что с ними можно сделать.

Посмотрю на уникальные не числовые значения в столбце user_score, так как у поля тип `object`, что странно для числового поля.

In [14]:
non_numeric_values = data['user_score'].apply(pd.to_numeric, errors='coerce').isna()
data.loc[non_numeric_values, 'user_score'].unique()

array([nan, 'tbd'], dtype=object)

Как видно из уникальных значений, в столбце есть значение 'tbd', которое означает "to be determined" - "будет определено".
Это значение означает, что у игры нет оценки пользователей, но она будет определена в будущем. Поэтому заменю это значение на пропуски.

In [15]:
data['user_score'] = data['user_score'].replace('tbd', np.nan)

Я вернусь к пропускам в user_score после того, когда преобразую типы данных.

In [16]:
missing_values(data)

,0
name,0.00%
platform,0.00%
year_of_release,0.00%
genre,0.00%
na_sales,0.00%
eu_sales,0.00%
jp_sales,0.00%
other_sales,0.00%
critic_score,45.74%
user_score,54.62%


Посмотрю на уникальные значения в столбце rating.

In [17]:
data['rating'].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

В столбце `rating` есть значение рейтинга ESRB за разные годы, того же K-A сейчас уже нет.
Также встречается значение 'RP', которое означает "Rating Pending" - "рейтинг ожидается".
Это значение означает, что у игры нет рейтинга, но он будет определён в будущем.
В целом, было бы логично заменить это значение на пропуски, но ESRB начала свою работу в 1994 году.
Поэтому сначала я преобразую `year_of_release` в числовой тип и посмотрю в каких годах рейтинга ESRB ещё не было, потому вернусь к заполнению пропусков в `rating`.

### Исправление типов данных

Ещё раз посмотрю на типы данных в датафрейме.

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16444 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     8922 non-null   float64
 9   user_score       7463 non-null   object 
 10  rating           9768 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


Преобразую `year_of_release` в целочисленный тип.

In [19]:
data['year_of_release'] = data['year_of_release'].astype('Int64')

Изменю тип данных в user_score на числовой тип.

In [20]:
data['user_score'] = data['user_score'].astype('float')

Проверю изменения в типах данных.

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16444 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     8922 non-null   float64
 9   user_score       7463 non-null   float64
 10  rating           9768 non-null   object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.5+ MB


`critic_score` и `user_score` имеют разный максимальный балл - 100 и 10 соответственно.
Нужно привести их к одному максимальному баллу, чтобы было удобнее сравнивать их.

In [23]:
data['user_score'] = data['user_score'] * 10

Проверю, что преобразование прошло успешно.

In [24]:
data.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,80.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,83.0,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,80.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


### Вернемся к пропускам в user_score и rating

Пропусков стало на \~14% больше, чем было до этого, но это и не удивительно.

Заполню пропуски в user_score медианным значением рейтинга пользователей по каждой игре.

In [22]:
data['user_score'] = data['user_score'].fillna(data.groupby('name')['user_score'].transform('median'))

Проверю, остались ли после этого пропуски в user_score.

В итоге пропусков в столбце user_score стало 48.16% - это много, но больше сделать с этими данными нечего, так как удалять их точно нельзя.
Пропуски тут опять же означают, что для остальных игр нет данных о рейтинге пользователей.

Описание проекта

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.
Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.
В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

Инструкция по выполнению проекта

Шаг 1. Откройте файл с данными и изучите общую информацию
путь к файлу: /datasets/games.csv

Шаг 2. Подготовьте данные.
Замените названия столбцов (приведите к нижнему регистру);
Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;
Обработайте пропуски при необходимости:
Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
Опишите причины, которые могли привести к пропускам;
Обратите внимание на аббревиатуру 'tbd' в столбцах с рейтингом. Отдельно разберите это значение и опишите, как его обработать;
Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.

Шаг 3. Проведите исследовательский анализ данных.
Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?
Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
Возьмите данные за соответствующий актуальный период. Актуальный период определите самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год.
Не учитывайте в работе данные за предыдущие годы.
Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.
Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.
Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы.
Соотнесите выводы с продажами игр на других платформах.
Посмотрите на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

Шаг 4. Составьте портрет пользователя каждого региона.
Определите для пользователя каждого региона (NA, EU, JP):
Самые популярные платформы (топ-5). Опишите различия в долях продаж.
Самые популярные жанры (топ-5). Поясните разницу.
Влияет ли рейтинг ESRB на продажи в отдельном регионе?

Шаг 5. Проверьте гипотезы.
Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;
Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.
Задайте самостоятельно пороговое значение alpha.
Поясните:
Как вы сформулировали нулевую и альтернативную гипотезы;
Какой критерий применили для проверки гипотез и почему.

Шаг 6. Напишите общий вывод.
Оформление: Задание выполните в Jupyter Notebook. Программный код заполните в ячейках типа code, текстовые пояснения — в ячейках типа markdown. Примените форматирование и заголовки.

Описание данных
Name — название игры
Platform — платформа
Year_of_Release — год выпуска
Genre — жанр игры
NA_sales — продажи в Северной Америке (миллионы долларов)
EU_sales — продажи в Европе (миллионы долларов)
JP_sales — продажи в Японии (миллионы долларов)
Other_sales — продажи в других странах (миллионы долларов)
Critic_Score — оценка критиков (максимум 100)
User_Score — оценка пользователей (максимум 10)
Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категори, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

Данные за 2016 год могут быть неполными.